In [543]:
import pandas as pd
import numpy as np
import re

In [544]:
df = pd.read_csv("/Users/santiago/ih-bootcamp/proyectos/project-pandas/GSAF5.csv")

# 1. Explorando la data

In [545]:
df.shape

(5992, 24)

In [546]:
df.head(5)
#df.tail(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [547]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case Number             5992 non-null   object
 1   Date                    5992 non-null   object
 2   Year                    5992 non-null   int64 
 3   Type                    5992 non-null   object
 4   Country                 5949 non-null   object
 5   Area                    5590 non-null   object
 6   Location                5496 non-null   object
 7   Activity                5465 non-null   object
 8   Name                    5792 non-null   object
 9   Sex                     5425 non-null   object
 10  Age                     3311 non-null   object
 11  Injury                  5965 non-null   object
 12  Fatal (Y/N)             5973 non-null   object
 13  Time                    2779 non-null   object
 14  Species                 3058 non-null   object
 15  Inve

In [548]:
# porcentaje de valores nulos por columna
df.columns=df.columns.str.replace(" ", "_")
df.columns=df.columns.str.lower()

In [549]:
nulls=list(zip(df.columns, [sum(df[x].isnull())/len(df[x])*100 for x in df]))

In [550]:
pd.DataFrame(nulls)

,0,1
0,case_number,0.000000
1,date,0.000000
2,year,0.000000
3,type,0.000000
4,country,0.717623
5,area,6.708945
6,location,8.277704
7,activity,8.795060
8,name,3.337784
9,sex_,9.462617


# Puntos a trabajar
    - Cambiar tipos de datos en variables de fecha a fecha
    - Tirar variables que no sean útiles y/o con muchos valores nulos
    - Unificar valores en variables como sex_, fatal_(y/n)
    - Limpiar valores de variable country para que sean sólo nombres de países, cambiar valores de regiones por NAs
    - Completar variables con muchos nulos con información de otra fuente.

## trabajando con los valores nulos

### Tirando unnamed:_23 y unnamed:_22

In [551]:
print(df["unnamed:_23"].unique(), df["unnamed:_22"].unique())

[nan 'Teramo' 'change filename'] [nan 'stopped here']


In [552]:
df[df["unnamed:_23"].isnull()!=True]

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
4182,1952.07.13,13-Jul-52,1952,Provoked,USA,California,"San Diego, San Diego County",Fishing,"Gerald Howard, on board sportsfishing boat Ter...",M,...,NaN,"L.A. Times, 7/14/1952",1952.07.13-Howard.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.07.13,1952.07.13,1811,NaN,Teramo
5585,1879.03.10,10-Mar-1879,1879,Invalid,AUSTRALIA,New South Wales,Near Sydney,The steamship Bonnie Dundee lost in collision,Cabin boy of the Bonnie Dundee,M,...,NaN,"Star, 3/22/1879",1879.03.10-Bonnie-Dundee.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1879.03.10,1879.03.10,408,NaN,change filename


In [553]:
df[df["unnamed:_22"].isnull()!=True]

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
1247,2006.06.18,18-Jun-06,2006,Unprovoked,BRAZIL,Pernambuco,"Punta Del Chifre Beach, Olinda",Body boarding,Humberto Pessoa Batista,M,...,NaN,globalsurfnews.com,2006.06.18-Batista.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.06.18,2006.06.18,4746,stopped here,NaN


In [554]:
#al parecer no tienen información relevante las columnas "unnamed:_22" y "unnamed:_23"
df.drop(columns=["unnamed:_22", "unnamed:_23","case_number.1", "case_number.2", "href_formula"], inplace=True)

## Trabajo por variables
### investigator_or_source 

In [555]:
df.investigator_or_source.nunique()

4752

In [556]:
set(df.investigator_or_source.unique())

{nan,
 'Mrs. Crewe, B. Lipschitz, M. Levine, GSAF',
 'New York Times, 11/28/1929',
 'J. E. Kirby, letter to Captain Dinning, dated 10/14/1927',
 'Sydney Daily Telegraph, no date; L. Schultz & M. Malin, p.548',
 'News-Journal, 8/11/1995',
 'Courier-Mail, 8/10/2010',
 'E.S. Hobson; SAF Case #955',
 'Naval Journal (undated)',
 'You Tube, Baby Blacktip Shark Attack - How NOT to hold a shark.',
 'L. A. Pereira & J. Morris; JC, 12/27/1999',
 'Washington Post, 6/30/1964; SAF Case #1295',
 'C. Creswell, GSAF; J. Johnson, Post and Courier, 9/26/2007',
 'R. Mizutani, KHON2, 9/23/2011',
 'Club of Mozambique',
 'Norhern Rivers Echo, 11/10/2014',
 'K. Nakaya; Orlando Sentinel, 3/20/1992, p.A10 & 6/18/1992, p. A3',
 'Geraldton Guardian and Express, 2/3/1930',
 'Global Post, 10/26/2013',
 'Daily Dispatch, 9/16/1998',
 'Sunday Mail, 10/29/1989',
 'Daytona Beach Morning Journal, 11/2/1974',
 'D. Lowe',
 'WPBF.com, 9/2/2014',
 'Honolulu Advertiser',
 'Palm Beach Post, 11/3/1990; Miami Herald, 11/2/1990 

### Country

In [557]:
df.country=df.country.str.lower()
df.country.unique()

array(['usa', 'australia', 'new caledonia', 'reunion', 'bahamas', 'spain',
       'china', 'japan', 'columbia', 'south africa', 'egypt',
       'new zealand', 'indonesia', 'french polynesia', 'cape verde',
       'fiji', 'brazil', 'dominican republic', 'cayman islands',
       'united arab emirates', 'aruba', 'mozambique', 'thailand',
       'puerto rico', 'italy', 'mexico', 'atlantic ocean', 'greece',
       'mauritius', nan, 'st. martin', 'france', 'ecuador',
       'papua new guinea', 'trinidad & tobago', 'kiribati', 'israel',
       'diego garcia', 'taiwan', 'jamaica', 'palestinian territories',
       'guam', 'seychelles', 'belize', 'philippines', 'nigeria', 'tonga',
       'scotland', 'canada', 'croatia', 'saudi arabia', 'chile',
       'antigua', 'kenya', 'russia', 'turks & caicos', 'costa rica',
       'united kingdom', 'malaysia', 'united arab emirates (uae)',
       'samoa', 'azores', 'solomon islands', 'south korea', 'malta',
       'vietnam', 'madagascar', 'panama', 'somali

In [558]:

df.country=[re.sub(r"(/ \w* \w*)|(/ \w*)|\?| \(\w* \w*\)", "", str(x)) for x in df.country]
df.country=[re.sub(r".*ocean.*", "", str(x)) for x in df.country]

# Casos específicos
df.country=df.country.replace("between portugal & india", "").replace("asia", "").replace("united arab emirates (uae)", "united arab emirates")
df.country=df.country.replace("gulf of aden", "yemen").replace("tasman sea", "australia")
df.country=df.country.replace("coast of africa", "").replace("mediterranean sea", "").replace("red sea", "egypt")


In [559]:
#casos que hay que revisar mas: "diego garcia"

## Type

In [560]:
# Unificando boating y boat para reducir una categoría
df.type.unique()
df.type=np.where(df.type=='Boating', 'Boat',df.type)

## Sex

In [561]:
df.sex_.unique()

array(['M', nan, 'F', 'M ', 'lli', 'N', '.'], dtype=object)

In [562]:
# Revisando y cambiando casos atípicos lli se cambia por M por el 
# nombre (se asume que se Brian se identifica como hombre).
df.sex_=df.sex_.replace('M ', 'M').replace("lli", "M").replace('.', 'nan').replace('N', 'nan')
df.sex_.value_counts()

M      4838
F       585
nan       2
Name: sex_, dtype: int64

## Activity

In [563]:
# Oh no, son muchas actividades diferentes. A continuación se intenta reducirlas lo más posible
df.activity.nunique()

1492

In [564]:
set(df.activity.unique())

{nan,
 'Crouching in the water',
 'Fishing from a small boat & put his hand in the water while holding a dead fish',
 'Yacht Gooney Bird foundered, 4 survivors on raft',
 'Clinging to shipwrecked junk',
 'Playing',
 'On boat, preparing to dive',
 'Petting a shark',
 'Steinhart Aquarium',
 'Coming ashore on a hawser',
 'Sea Disaster, wreck of the  SS Norwich City',
 'Spearfishing, Scuba diving',
 'Pulling hooked salmon to boat',
 'Swimming / jumping off a jetty',
 'Taking wife to beach & about 1 m from the shore',
 'Playing / jumping',
 'Diving for crayfish',
 'Dived into a school of baitfish',
 'Fishing from surfski',
 'Scuba diving',
 'Fishing on a boat',
 'Surfing, paddling shorewards',
 'Boat with 5 men capsized while returning to the Amerique',
 'Paddle Boarding',
 'Swimming behind sailboat',
 'Fell onto dead shark',
 'Free diving, hunting crayfish',
 'Sleeping in anchored boat',
 'Jumped overboard from Norwegian steamship Venator',
 'Air disaster: crash of Air France Super Constel

In [565]:
df.activity=df.activity.str.lower()
df.activity=[re.sub(r".*fishing.*|.*swimming.*|.*diving.*|.*surf.*|.*bathing.*|.*fell overvoard.*|.*filming.*|.*floating.*", "", str(x)) for x in df.activity]
df.activity=[re.sub(r".*wreck.*|.*sinking.*|.*boat.*|.*sea disaster.*|.*capsized.*|.*sank.*|.*collided.*|.*sunk.*", "boat wreck", str(x)) for x in df.activity]
df.activity=[re.sub(r".*dived.*", "diving", str(x)) for x in df.activity]
df.activity=[re.sub(r".*fell.*|.*jump.*", "fell/jumped into water", str(x)) for x in df.activity]
df.activity=[re.sub(r".*padling.*|.*rowing.*|.*sailing.*|.*canoeing.*|.*paddling.*|.*paddle.|.*yacht.*", "cruising", str(x)) for x in df.activity]
df.activity=[re.sub(r".*snorkeling.*", "swimming", str(x)) for x in df.activity]
df.activity=[re.sub(r".*adrift.*", "being adrift", str(x)) for x in df.activity]
df.activity=[re.sub(r".*air disaster.*|.*aircaft exploded.*|.*aircraft crashed.*|.*b-24.*|.*boeing.*", "air disaster", str(x)) for x in df.activity]
df.activity=[re.sub(r".*hunting.*|.*netting.*|.*killing.*|.*collecting.*", "fishing", str(x)) for x in df.activity]
df.activity=[re.sub(r".*standing.*|.*wading.*|.*walking.*", "standing/wading in water", str(x)) for x in df.activity]




In [566]:
# Se redujo a cerca de un cuarto de las actividades originales y se podría continuar reduciendo
df.activity.nunique()

371

## Year

In [567]:
df.date.nunique()

5128

In [568]:
set(df.date.unique())

{'02-Nov-05',
 'Reported 08-Jul-1908',
 '28-Apr-88',
 '18-Oct-12',
 '18-Dec-57',
 '25-Apr-15',
 '15-Jan-54',
 '11-Apr-03',
 '13-Feb-48',
 '17-Jul-88',
 '19-Jul-89',
 '26-May-77',
 '05-Jul-67',
 '18-Feb-62',
 '25-Dec-13',
 '21-Nov-98',
 '10-Dec-48',
 '23-Feb-02',
 '28-Jul-56',
 '03-Dec-1882',
 '23-Oct-34',
 'Reported 24-May-1944',
 '18-Jun-14',
 'Dec-93',
 '21-Sep-02',
 '17-Nov-00',
 '08-Aug-1899',
 '30-Mar-63',
 '25-Jul-01',
 '1883',
 'Reported 12-Sep-1894',
 '01-Jul-91',
 '24-Sep-01',
 '26-Mar-15',
 '15-Mar-02',
 '22-Jun-05',
 '05-May-81',
 'Apr-93',
 '24-Dec-56',
 '16-Jan-13',
 '11-Nov-06',
 '10-Feb-31',
 '19-Jun-00',
 '04-May-01',
 '09-May-67',
 'Reported 31-Aug-1959',
 '06-Jun-11',
 '30-Jul-93',
 '25-Jan-06',
 '09-Apr-59',
 '24-Oct-13',
 '01-Aug-05',
 '25-Sep-00',
 '31-Dec-11',
 'Reported 29-Mar-2011',
 '23-Jul-06',
 '05-Feb-70',
 '16-Jul-53',
 'Nov-67',
 '19-Jul-71',
 '29-Nov-64',
 'Ca. 1903',
 '19-Jun-65',
 '28-Aug-33',
 '10-Nov-15',
 '30-Jan-02',
 '18-Dec-07',
 '29-Jan-16',
 '09

In [569]:
df.date=df.date.str.lower()
df.date=df.date.str.replace("a few years before 1938", "1938")
df.date=[re.sub(r"no date, before |between|between   |reported |no date, before |before |circa |beforer |early |         |   |\?|reported ", "", str(x)) for x in df.date]
df.date=[re.sub(r"ca. |late| aug-1951 &  |during the war|summer of |mar-1943 to |letter dated| reported| 18 & |sep-1942 to |^ *", "", str(x)) for x in df.date]
df.date=[re.sub(r'summer |decp|1990 or |nov- or|may & |mar-1925 or |jul- to |mid |nov-1909 to |anniversary day" jan-1850 or ', "", str(x)) for x in df.date]
df.date=[re.sub(r' \(same day as  1954.00.00.f\)|reportd |1950 - |1920 -', "", str(x)) for x in df.date]
df.date=[re.sub(r"no date.*|.*b.b..*", "nan", str(x)) for x in df.date]

In [570]:
#para simplificar tomaré sólo mes y año de fechas
df.date=[re.sub(r"\d\d-", "", str(x)) for x in df.date]

#para estar seguro del año se usa la columna year para pegáreselo a las fechas en los caso en que aplique
df['date_formato'] = [re.search(r"\w{3}-\d{2}$", str(x))!=None for x in df.date]
nueva_date=[]
for i in range(0, len(df.date)):
    if df['date_formato'][i]==True:
        nueva_date.append(re.sub(r"\d\d$","", df['date'][i])+str(df['year'][i]))
    else:
        nueva_date.append(df['date'][i])
df['date']=nueva_date

In [571]:
df.drop(columns=['date_formato', 'original_order'], inplace=True)

In [572]:
df[df.date=="jun-1886"]

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,pdf,href
5507,1886.06.17,jun-1886,1886,Unprovoked,australia,Queensland,Off Bribie Heads,,Lacoon,M,18,FATAL,Y,NaN,NaN,"Brisbane Courier, 6/21/1886",1886.06.17-Lacoon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
5508,1886.06.02,jun-1886,1886,Invalid,usa,Hawaii,"Hamakua Homakua, Hawai'i",,2 women,F,NaN,The body of one woman had been bitten by a sha...,Y,NaN,NaN,G. H. Balazs & A. H. Kam; V.M. Coppleson (1958...,1886.06.02-Hamakua.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [578]:
# las fechas que solo son año le agregamos junio como mes
df.date=[re.sub(r"\d\d-", "", str(x)) for x in df.date]
df['date_formato'] = [re.search(r"^\d{4}$", str(x))!=None for x in df.date]

for x in range(0,len(df.date)):
    if df['date_formato'][x]==True:
        df["date"][x]="jun-"+str(df.date[x])

## Fatal

In [574]:

df['fatal_(y/n)'].unique()
df['fatal_(y/n)']=df['fatal_(y/n)'].str.lower()
df['fatal_(y/n)']=df['fatal_(y/n)'].replace(' n', "n").replace('n ', 'n')

In [575]:
df['fatal_(y/n)'].unique()

array(['n', 'y', nan, 'unknown', 'f', '#value!'], dtype=object)

In [576]:
df.loc[(df['fatal_(y/n)'] =="f")|(df['fatal_(y/n)'] =="unknown")|(df['fatal_(y/n)'] =='#value!')] = "nan"
df['fatal_(y/n)']=df['fatal_(y/n)'].replace('n', False).replace('y', True)
df['fatal_(y/n)'] = df['fatal_(y/n)'].astype('bool')
df.rename(columns={'fatal_(y/n)':'fatal'}, inplace=True)

In [577]:
df['fatbal'].unique()

array([False,  True])

## Name

In [585]:
df.name=df.name.str.lower()
df.name=df.name.replace("male", "").replace("male x 2", "").replace("5 men", "")
set(df.name.unique())

{'',
 nan,
 'robert williams',
 'laurent borgna ',
 'barbara green',
 'fernando licay',
 'giovanni picarelli',
 'dr. j. d. morgan',
 'george (or edward) eaton',
 'captain thaxton',
 'cole',
 'charles cook',
 'emmanuel nativel',
 'dylan marks',
 'raymond t. gold',
 'sam myer',
 'king scherer',
 'davies',
 'helga pogl',
 'george parkin',
 'charles swan, a returned soldier',
 'andrew vance',
 'joel tatro',
 'blake perry',
 'a old fisherman',
 'pierre de rotalier',
 'bruce corby',
 'alexis huesgen',
 'small speedboat, occupan: paul mcnally',
 'naval aviator',
 'stephen johnson',
 'sandra fletcher',
 'sergi zaloukaev',
 'josia mcspadden',
 'adam metz',
 'johan loekstad, a norwegian seaman',
 'john lear',
 'matthew smithers',
 'nunley',
 'willaim browne',
 'ian watkins',
 'diego gomes mota',
 'dakota hatfield',
 'richard chung',
 'roberto perez',
 'kath. o�neill',
 'c. stokes',
 'kim pearce',
 'howard rosen',
 'noel cross',
 'pablo medina',
 'albert mainlander, an aboriginal',
 'william mcfl